In [1]:
import pip

package_names = ['awswrangler'] 
pip.main(['install'] + package_names + ['--upgrade'])

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


0

In [2]:
# Nativos
from dateutil.relativedelta import relativedelta
from time import gmtime, strftime
from datetime import datetime
import random as rn
import joblib
import json
import sys
import os
import gc

#nube
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker import get_execution_role
import awswrangler as wr
import sagemaker
import boto3

#calculo
import pandas as pd
import numpy as np
import scipy

#grafico
from IPython.display import display
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid")

#Interacciones con output
import warnings
warnings.filterwarnings("ignore")
# warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

gc.collect()
# MODELS
#from lightgbm import LGBMClassifier

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)
#import scorecardpy as sc
#from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

#from utils import *
#from modeler import *
#subfolder = "data"
# os.listdir(subfolder)

from setup import *

////////////////////////////////////////////////// 

path_container_input:  /opt/ml/processing/input
path_container_output:  /opt/ml/processing/output
path_container_utils:  /opt/ml/processing/input/utils
path_container_universo:  /opt/ml/processing/input/HM_UNIVERSO_PROPENSION_DESEMBOLSO_BPE
target :::::::::  target_desembolso_f2m_mayor_30_menor_180


In [3]:
# imports
from UTILITARIO_CODE.utils import targets
from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.amazon.amazon_estimator import get_image_uri
import awswrangler as wr
import pandas as pd
import sagemaker
import boto3

print("bucket: ", bucket)
print("region: ", region)
print("now: ", now)
print("sufijo: ", sufijo)

bucket:  sagemaker-us-east-1-058528764918
region:  us-east-1
now:  2021-09-27 18:57:04.912907
sufijo:  target_desembolso_f2m_mayor_30_menor_180_clasif_normal


In [4]:
print(len(seleccionadas))
print(seleccionadas)

19
['deuda_sf_prom_ult9m', 'saldo_coloc_direct_tc', 'prom_reprog_u12m', 'ult_var_saldo_ajustado_amt', 'nro_entid_financ_prom_ult9m_cnt', 'monto_adquirido_u6_amt', 'nro_var_10k_30k_negativa_u6', 'var_neta_saldo_ajustado_u3_amt', 'prom_gar_u12m', 'nroregs_reactiva_bcos', 'monto_pagado_u3_amt', 'saldo_coloc_direct_vig_bcos', 'saldo_coloc_direct_vig_cmpt', 'nro_entidades', 'monto_pagado_ult_rcc_amt', 'tendencia_nro_coloc_direct_bancos_v2', 'sow_cajas', 'prom_fae_u12m', 'nro_var_10k_30k_negativa_u3']


### Load data

In [5]:
print("uri_output: ", uri_output)
print("uri_contraste: ", uri_contraste)
print("uri_predict: ", uri_predict)

uri_output:  s3://sagemaker-us-east-1-058528764918/vpc/propension/output3
uri_contraste:  s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste
uri_predict:  s3://sagemaker-us-east-1-058528764918/vpc/propension/prediccion


** OJO: modificar esta parte en caso de cambios de periodos**

In [6]:
#uri_validation_202102 = uri_contraste + '/202102_{}.csv'.format(sufijo)
uri_validation_202103 = uri_contraste + '/202103_{}.csv'.format(sufijo)
uri_validation_202104 = uri_contraste + '/202104_{}.csv'.format(sufijo)
uri_validation_202105 = uri_contraste + '/202105_{}.csv'.format(sufijo)
uri_validation_202106 = uri_contraste + '/202106_{}.csv'.format(sufijo)
uri_validation_202107 = uri_contraste + '/202107_{}.csv'.format(sufijo)

#print(uri_validation_202102)
print(uri_validation_202103)
print(uri_validation_202104)
print(uri_validation_202105)
print(uri_validation_202107)

s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202103_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202104_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202105_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv
s3://sagemaker-us-east-1-058528764918/vpc/propension/contraste/202107_target_desembolso_f2m_mayor_30_menor_180_clasif_normal.csv


In [7]:
#_2006 = wr.s3.read_csv(
#    uri_validation_202106, 
#    sep=',',
#    dtype={'cod_sbs_val': str}
#)

#_2006.head()

In [8]:
# _2006.shape

### FEEDBACK

In [9]:
campania_ibk = pd.read_csv(
    'resultados_preaprobados_aporbados_5.csv', 
    dtype={'PERIODO': str, 'NUM_RUC': str, 'COD_SBS': str, 'CODSBS': str, 'COD_UNICO': str}
)
campania_ibk.head()

,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
0,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202109,Nuevo Aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,NaN,1.0,NaN,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,PN,78.0,NaN,NaN,Nuevo Aprobado,No Acepta Campaña,D.NO ACEPTA,D.NO ACEPTA,TLV,NaN,10,202109,0120556819
1,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202106,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Fresco,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,56.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,NaN,NaN,10,202106,0120556819
2,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202103,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0120556819
3,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,1,0,1,183875.0,202102,Nuevo Pre-aprobado,10004003573,0012854893,No Acepta Campaña,No necesita campaña,"Cliente no desea créditos, no dio mayor detall...",2021-02-19 16:04:00,GESTIONADO,contactado,NaN,Sin desembolso,0,1,NaN,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,24.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202102,0120556819
4,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202107,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,NaN,Repetido,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,63.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202107,0120556819


In [10]:
campania_ibk['CAMPANHA'].value_counts()

Nuevo Pre-aprobado          203035
Ecosistemas Corporativos    122399
Nuevo Aprobado               81699
Giro de Línea                15011
Recurrente Reactiva CEM      11453
Recurrente Ágil CEM           5835
Recurrente Aprobado           2061
Name: CAMPANHA, dtype: int64

In [11]:
campania_ibk.isnull().sum()

CODSBS                      29898
TIPDOC                      29898
NOMBRE_COMPLETO             30503
TIPPERSONA                  29898
FLG_ACEPTA                      0
FLG_GESTIONADO                  0
FLG_DESEMBOLSADO                0
FLG_CONTACTADO                  0
ID_EMPRESA                     97
PERIODO                         0
CAMPANHA                        0
NUM_RUC                         0
COD_UNICO                  214099
GESTION_TIPO_2             352745
GESTION_SUBTIPO            354436
GESTION_COMENTARIO         357760
GESTION_FECHA_REGISTRO     352876
GESTIONADO                      0
Contactados                     0
Acepta                     434393
Desembolsado                    0
CTD_DESEMBOLSO                  0
CTD_DESPLEGADOS                 0
COLOCACION_NETA            437076
FRESCURA_LEAD                   0
OFERTA_APROBADA             46477
flg_tlv                    251812
Acepta_EN                  434722
Acepta_tlv                 254528
Acepta_tlv_sin

In [12]:
campania_ibk[campania_ibk['COD_SBS'].isnull()].groupby(by=['PERIODO']).size()

PERIODO
202103       1
202104     114
202105    2250
202106    3680
202107    4208
202108    9738
202109    9907
dtype: int64

In [13]:
print(campania_ibk.shape)
campania_ibk = campania_ibk[campania_ibk['COD_SBS'].notnull()]
campania_ibk['COLOCACION_NETA'] = campania_ibk['COLOCACION_NETA'].fillna(0)                        
print(campania_ibk.shape)

campania_ibk.head()

(441493, 49)
(411595, 49)


,CODSBS,TIPDOC,NOMBRE_COMPLETO,TIPPERSONA,FLG_ACEPTA,FLG_GESTIONADO,FLG_DESEMBOLSADO,FLG_CONTACTADO,ID_EMPRESA,PERIODO,CAMPANHA,NUM_RUC,COD_UNICO,GESTION_TIPO_2,GESTION_SUBTIPO,GESTION_COMENTARIO,GESTION_FECHA_REGISTRO,GESTIONADO,Contactados,Acepta,Desembolsado,CTD_DESEMBOLSO,CTD_DESPLEGADOS,COLOCACION_NETA,FRESCURA_LEAD,OFERTA_APROBADA,flg_tlv,Acepta_EN,Acepta_tlv,Acepta_tlv_sin_contacto,BARRIDO,CE,INTERESADO,Pasa_pre_filtro,PASA_OFERTA,ENVIADO_GTP,TIPO_PERSONA,ID_CAMPANHA_INSTANCIA,canal,registro_ejecutivo,SUB_CAMPANHA,GESTION_TIPO_TLV,GESTION_TLV_NIVEL_1,GESTION_TLV_NIVEL_2,CANAL_INICIAL,FLG_REPROGRAMACION,LETTER,PERIODO_CAMPANHA,COD_SBS
0,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202109,Nuevo Aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,NaN,1.0,NaN,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,PN,78.0,NaN,NaN,Nuevo Aprobado,No Acepta Campaña,D.NO ACEPTA,D.NO ACEPTA,TLV,NaN,10,202109,0120556819
1,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202106,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Fresco,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,56.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.TELEFONO ERRADO,A.TELEFONO ERRADO,NaN,NaN,10,202106,0120556819
2,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202103,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,150000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,28.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202103,0120556819
3,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,1,0,1,183875.0,202102,Nuevo Pre-aprobado,10004003573,0012854893,No Acepta Campaña,No necesita campaña,"Cliente no desea créditos, no dio mayor detall...",2021-02-19 16:04:00,GESTIONADO,contactado,NaN,Sin desembolso,0,1,0.0,Fresco,120000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PN,24.0,NaN,NaN,Nuevo Pre-aprobado,NaN,NaN,NaN,NaN,NaN,10,202102,0120556819
4,0120556819,1.0,CALIZAYA TICONA VIRGINIA MARIA\r,1.0,0,0,0,0,183875.0,202107,Nuevo Pre-aprobado,10004003573,0012854893,NaN,NaN,NaN,NaN,Sin gestion,sin contacto,NaN,Sin desembolso,0,1,0.0,Repetido,150000.0,1.0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,PN,63.0,NaN,NaN,Nuevo Pre-aprobado,Sin Contacto,A.NO CONTACTO,A.NO CONTACTO,TLV,NaN,10,202107,0120556819


In [14]:
from sklearn.metrics import roc_auc_score

In [15]:
def get_score_campania(mes_campania, data_rcc):
    camapania_target = campania_ibk[
        (campania_ibk['PERIODO'] == mes_campania)
    ]
    print(camapania_target.shape, camapania_target['PERIODO'].unique())
    
    try:
        print("-"*25)
        print("AUC BPE RCC:     ", roc_auc_score(data_rcc[target], data_rcc['score_desembolso']))
    except:
        pass
    
    camapania_target['COD_SBS'] = camapania_target['COD_SBS'].astype(int).astype(str)
    data_rcc['cod_sbs_val'] = data_rcc['cod_sbs_val'].astype(int).astype(str)
    
    validation_campania = data_rcc[data_rcc['cod_sbs_val'].isin(list(camapania_target['COD_SBS']))]
    try:
        roc_auc_campania = roc_auc_score(validation_campania[target], validation_campania['score_desembolso'])
        print("AUC CAMPANIA RCC: ", roc_auc_campania, roc_auc_campania*2-1)
    except:
        pass
    
    print("validation_campania: ", validation_campania.shape)
    validation_campania = validation_campania.merge(
        camapania_target, how='left', left_on='cod_sbs_val' , right_on ='COD_SBS'
    )
    print("camapania_target: ", camapania_target.shape)
    print("validation_campania: ", validation_campania.shape)
    
    roc_auc_acepta = roc_auc_score(validation_campania['FLG_ACEPTA'], validation_campania['score_desembolso'])
    print("AUC ACEPTA:      ",roc_auc_acepta, roc_auc_acepta*2-1)

    roc_auc_desembolsa = roc_auc_score(validation_campania['FLG_DESEMBOLSADO'], validation_campania['score_desembolso'])
    print("AUC DESEMBOLSA: ", roc_auc_desembolsa, roc_auc_desembolsa*2-1)
    
    group_ = validation_campania.groupby(by=['GRUPO_PROPENSION']).agg({
        target: ['count', 'sum', 'mean'], 
        'score_desembolso': 'mean',
        'FLG_ACEPTA': ['mean', 'sum'],
        'FLG_DESEMBOLSADO': ['mean', 'sum']
    })
    
    return group_, validation_campania

### 202102 ---> CAMPAÑA 202104

In [16]:
#_2002 = wr.s3.read_csv(
#    uri_validation_202102, 
#    sep=',',
#    dtype={'cod_sbs_val': str}
#)
#_2002.head(3)

In [17]:
#group_202104, campania_bpe_202104 = get_score_campania('202104', _2002)
#group_202104

In [18]:
#del _2002
#campania_bpe_202104['GRUPO_PROPENSION'].value_counts(normalize=True)*100

### 202103 ---> CAMPAÑA 202105

In [19]:
_2003 = wr.s3.read_csv(
    uri_validation_202103, 
    sep=',',
    dtype={'cod_sbs_val': str}
)
_2003.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,48759904,0.0,6077.54,0.0,4558.153333,-616.17,1,0.0,0,-1794.37,0.00,0,1794.37,0.0,0.0,1,616.17,0.000000,1.0,0.000,0,0.007450,I
1,149898271,0.0,24525.75,0.0,17737.965000,900.00,1,900.0,1,-23266.88,2.25,0,24166.88,0.0,0.0,1,0.00,0.333333,1.0,0.000,1,0.030875,C
2,126112726,0.0,7000.35,0.0,0.000000,0.00,1,0.0,0,0.00,1.00,0,0.00,0.0,0.0,1,0.00,1.000000,0.0,5833.625,0,0.007600,I


In [20]:
group_202105, campania_bpe_202105 = get_score_campania('202105', _2003)
group_202105

(48220, 49) ['202105']
-------------------------
AUC BPE RCC:      0.7720062554415492
AUC CAMPANIA RCC:  0.7044034429336313 0.40880688586726266
validation_campania:  (35925, 23)
camapania_target:  (48220, 49)
validation_campania:  (37476, 72)
AUC ACEPTA:       0.6667767095782324 0.33355341915646486
AUC DESEMBOLSA:  0.6960890792394412 0.3921781584788824


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA      FLG_DESEMBOLSADO     
                                                    count    sum      mean             mean       mean  sum             mean  sum
GRUPO_PROPENSION                                                                                                                 
A                                                    5348  646.0  0.120793         0.093336   0.063949  342         0.048429  259
B                                                    4995  273.0  0.054655         0.047192   0.024424  122         0.017818   89
C                                                    4572  197.0  0.043088         0.033542   0.021872  100         0.010717   49
D                                                    4318  157.0  0.036359         0.025283   0.015980   69         0.008800   38
E                                                    4470  106.0  0.023714         0.020022   0.015884   71         0.008054   36
F                                                    5236  100.0  0.019099         0.015568   0.013560   71         0.007257   38
G                                                    3582   76.0  0.021217         0.012575   0.015355   55         0.009492   34
H                                                    2818   50.0  0.017743         0.010063   0.012065   34         0.008517   24
I                                                    1448   22.0  0.015193         0.007750   0.009669   14         0.006215    9
J                                                     689   11.0  0.015965         0.005396   0.017417   12         0.013062    9

In [21]:
del _2003
campania_bpe_202105['GRUPO_PROPENSION'].value_counts(normalize=True)*100

NumExpr defaulting to 4 threads.


A    14.270466
F    13.971608
B    13.328530
C    12.199808
E    11.927634
D    11.522041
G     9.558117
H     7.519479
I     3.863806
J     1.838510
Name: GRUPO_PROPENSION, dtype: float64

### 202104 ---> CAMPAÑA 202106

In [22]:
_2004 = wr.s3.read_csv(
    uri_validation_202104, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2004.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION
0,203570678,0.0,80314.40,0.0,0.0000,-9379.89,0,250000.00,0,231724.84,0.000000,0,18275.16,231724.84,231724.84,1,9379.89,0.0,0.000,0.000000,0,0.031269,C
1,183894013,0.0,25441.24,0.0,0.0000,0.00,0,0.00,0,0.00,0.000000,1,0.00,0.00,0.00,1,0.00,1.0,0.000,0.000000,0,0.006905,I
2,74279325,0.0,99463.00,0.0,81768.9575,-1384.21,1,10225.52,0,-4265.70,0.833333,0,4265.70,8600.59,0.00,2,1384.21,1.0,0.883,18727.464167,0,0.033284,C


In [23]:
group_202106, campania_bpe_202106 = get_score_campania('202106', _2004)
group_202106

(48496, 49) ['202106']
-------------------------
AUC BPE RCC:      0.7526423472331193
AUC CAMPANIA RCC:  0.704390766517793 0.40878153303558595
validation_campania:  (36089, 23)
camapania_target:  (48496, 49)
validation_campania:  (36441, 72)
AUC ACEPTA:       0.6720485223105532 0.34409704462110646
AUC DESEMBOLSA:  0.7262134244631651 0.4524268489263301


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA      FLG_DESEMBOLSADO     
                                                    count    sum      mean             mean       mean  sum             mean  sum
GRUPO_PROPENSION                                                                                                                 
A                                                    5470  647.0  0.118282         0.090879   0.063620  348         0.048995  268
B                                                    5027  313.0  0.062264         0.046688   0.032027  161         0.018699   94
C                                                    4386  206.0  0.046968         0.033229   0.025764  113         0.014364   63
D                                                    4249  129.0  0.030360         0.025104   0.016004   68         0.007531   32
E                                                    4489  128.0  0.028514         0.019903   0.018490   83         0.010247   46
F                                                    4793   95.0  0.019821         0.015496   0.015439   74         0.005633   27
G                                                    3428   75.0  0.021879         0.012602   0.016919   58         0.007585   26
H                                                    2331   43.0  0.018447         0.010033   0.012012   28         0.006006   14
I                                                    1482   17.0  0.011471         0.007740   0.008097   12         0.004049    6
J                                                     786    5.0  0.006361         0.005332   0.005089    4         0.002545    2

In [24]:
del _2004
campania_bpe_202106['GRUPO_PROPENSION'].value_counts(normalize=True)*100

A    15.010565
B    13.794901
F    13.152767
E    12.318542
C    12.035894
D    11.659943
G     9.406987
H     6.396641
I     4.066848
J     2.156911
Name: GRUPO_PROPENSION, dtype: float64

### 202105 ---> CAMPAÑA 202107

In [25]:
_2005 = wr.s3.read_csv(
    uri_validation_202105, 
    sep=',',
    dtype={'cod_sbs_val': str}
)

_2005.head(3)

,cod_sbs_val,target_desembolso_f2m_mayor_30_menor_180,deuda_sf_prom_ult9m,saldo_coloc_direct_tc,prom_reprog_u12m,ult_var_saldo_ajustado_amt,nro_entid_financ_prom_ult9m_cnt,monto_adquirido_u6_amt,nro_var_10k_30k_negativa_u6,var_neta_saldo_ajustado_u3_amt,prom_gar_u12m,nroregs_reactiva_bcos,monto_pagado_u3_amt,saldo_coloc_direct_vig_bcos,saldo_coloc_direct_vig_cmpt,nro_entidades,monto_pagado_ult_rcc_amt,tendencia_nro_coloc_direct_bancos_v2,sow_cajas,prom_fae_u12m,nro_var_10k_30k_negativa_u3,score_desembolso,GRUPO_PROPENSION,GRUPO_PROPENSION_FINO
0,30419162,0.0,59487.23,0.0,0.000000,-2558.09,1,11789.22,0,7862.12,2.155599e+04,1,3927.10,43772.33,0.00,1,2558.09,1.000,0.000,0.0,0,0.033920,C,E
1,75300956,0.0,589446.28,0.0,328314.754167,-22543.07,2,633912.87,1,122979.29,1.162966e+06,0,510933.58,629297.28,629297.28,3,22543.07,0.875,0.000,0.0,1,0.077572,A,C
2,69166024,0.0,33044.68,0.0,15639.696667,-958.23,2,0.00,0,-3224.89,2.000000e+00,0,3224.89,28798.82,0.00,2,958.23,1.000,0.001,0.0,0,0.008842,I,I


In [26]:
group_202107, campania_bpe_202107 = get_score_campania('202107', _2005)
group_202107

(53274, 49) ['202107']
-------------------------
AUC BPE RCC:      0.76016894021159
AUC CAMPANIA RCC:  0.7069714593471688 0.4139429186943375
validation_campania:  (39745, 24)
camapania_target:  (53274, 49)
validation_campania:  (40295, 73)
AUC ACEPTA:       0.6678247822326668 0.33564956446533367
AUC DESEMBOLSA:  0.7286777877350796 0.45735557547015926


target_desembolso_f2m_mayor_30_menor_180                  score_desembolso FLG_ACEPTA      FLG_DESEMBOLSADO     
                                                    count    sum      mean             mean       mean  sum             mean  sum
GRUPO_PROPENSION                                                                                                                 
A                                                    5799  795.0  0.137093         0.095405   0.063114  366         0.050871  295
B                                                    5124  374.0  0.072990         0.048336   0.026737  137         0.017955   92
C                                                    4557  237.0  0.052008         0.034320   0.017555   80         0.010094   46
D                                                    4322  176.0  0.040722         0.025813   0.012726   55         0.007867   34
E                                                    4766  166.0  0.034830         0.020368   0.014058   67         0.005455   26
F                                                    5467  153.0  0.027986         0.015795   0.016279   89         0.008231   45
G                                                    4241   89.0  0.020986         0.013156   0.014619   62         0.008017   34
H                                                    2864   58.0  0.020251         0.010338   0.015712   45         0.008380   24
I                                                    2099   33.0  0.015722         0.007815   0.011910   25         0.004288    9
J                                                    1056   10.0  0.009470         0.005496   0.006629    7         0.002841    3

### Otros resultados

#### ENOQUE DE ENTRENAMIENTO ANTIGUO